DATAFRAME->its like a SQL table

In [65]:
import pandas as pd
import numpy as np
from unidecode import unidecode


df = pd.DataFrame([[1,2,3],
                   [4,5,6],
                   [7,8,9]] , columns=['A','B','C'] , index=['x','y','z'])
df['A'].unique() #UNIQUE ELEMENTS OF COLUMN 'A'


array([1, 4, 7])

IMPORTING FILES INTO DATAFRAMES

In [43]:
bios2 = pd.read_csv('https://raw.githubusercontent.com/KeithGalli/complete-pandas-tutorial/refs/heads/master/data/bios.csv')
coffee = pd.read_csv('https://raw.githubusercontent.com/KeithGalli/complete-pandas-tutorial/refs/heads/master/warmup-data/coffee.csv')
bios = pd.read_csv('https://raw.githubusercontent.com/KeithGalli/complete-pandas-tutorial/refs/heads/master/data/bios.csv')

MODIFYING COLUMN NAMES

In [ ]:
coffee.rename(columns={'Day':'day' , 'units_sols': 'units sold' , 'Coffee Type':'coffee type'} , inplace=True) #or coffee.columns = coffee.columns.set.lower()
coffee.columns = coffee.columns.str.replace('_',' ')
coffee

ACCESSING DATA IN PANDAS

In [ ]:
coffee.sample(10)

coffee.loc[1] #loc -> access by row number
coffee.loc[[0,1,5]]

#SLICING IN LOC ans ILOC (this is also rows,column ... but columns have to be named)
coffee.loc[5:8,['day','units sold']] #in iloc this will be [5:9(in iloc the end point is exclusive) , [0,2]]

#by column name
coffee['units sold']
e
#sorting
coffee.sort_values('units sold')
coffee.sort_values('units sold' , ascending=False)
coffee.sort_values(['units sold','coffee type'] , ascending=[1,0]) #sort by units sold (ascending=true) first, in case they are same then look at coffee type(ascending=false)

#NOTE: INPLACE MEANS MODIFYING THE EXISTING DF, ELSE A NEW DF IS CREATED
#NOTE: SORTING DOES NOT REARRANGE THE INDEXES, TO ACHIEVE THAT:-
coffee2 = coffee.sort_values(by='day').reset_index(drop=True) #coffee 2 is the modified DF , if you wanna modify the original DF do coffee=
#NOTE: SIMPLY WRITING coffee.sort_values(by='day').reset_index(drop=True) WILL DO NOTHING

FILTERING DATA

In [ ]:
import unicodedata
bios.loc[(bios['height_cm']>200) & (bios['weight_kg']>100)]
bios.loc[(bios['height_cm']>200) & (bios['weight_kg']>100) , ['name','height_cm','weight_kg']] #cause in the end its .loc[row,column]

bios.loc[ (bios['height_cm']>200) & (bios['weight_kg']>100) & (bios['born_country'] == 'USA') , ['name','height_cm','weight_kg','born_country']] #multiple conditions

bios[bios['name'].str.contains('shaq', case = False) ] #case sensitive by default


#SMALL CLEANING EXAMPLE ->ARRANGING NAMES COLUMN IN ALPHABETIC ORDER


In [5]:
from unidecode import unidecode
bios2 = (bios2.sort_values(
    by='name',
    key=lambda x: x.apply(unidecode).str.lower() 
).reset_index(drop=True) #this line rearranges the rows as well (mentioned above)
)

What problem this solves (context)
Your name column contains:-
  accented letters (José)
  Cyrillic look-alikes (Аlex)
  mixed casing (alex, Alex)
Normal sorting uses Unicode order, which gives weird results.

key=... → transform before sorting
key tells Pandas:
“Before comparing values, transform them using this function.”
Important:
The transformation is temporary :-
  is used only to compare values
  is NOT stored
  does NOT replace the column  
  exists only during sorting

x.apply(unidecode) -> treats è = e and à = a etc
.str.lower() ->treats alex, Alex, ALEX same

#ADDING/REMOVING COLUMNS

In [51]:
coffee['price'] = 4.99 #add column with default value
coffee['new price'] = np.where(coffee['Coffee Type'].str.lower() == 'espresso' , 5.99 , 4.99) #conditional assignment

#METHOD2
conditions = [
    coffee['Coffee Type'].str.lower() == 'espresso',
    coffee['Coffee Type'].str.lower() == 'latte'
]
prices = [5.99 , 6.99]
coffee['new price'] = np.select(conditions , prices , default=4.99)

coffee.drop(columns=['price'], inplace=True) #drop column

#METHOD3 = conditional adressing without NUMPY 
coffee.loc[coffee['Coffee Type'] == 'Espresso' , 'price'] = 5.99
coffee.loc[coffee['Coffee Type'] == 'Latte' , 'price'] = 6.99
coffee.drop(columns=['new price'], inplace=True)
coffee

,Day,Coffee Type,Units Sold,price
0,Monday,Espresso,25,5.99
1,Monday,Latte,15,6.99
2,Tuesday,Espresso,30,5.99
3,Tuesday,Latte,20,6.99
4,Wednesday,Espresso,35,5.99
5,Wednesday,Latte,25,6.99
6,Thursday,Espresso,40,5.99
7,Thursday,Latte,30,6.99
8,Friday,Espresso,45,5.99
9,Friday,Latte,35,6.99


if you wanna make a copy of a df and modify it without changing the original df, this is the method:-

In [ ]:
#coffee_new = coffee.copy() #coffee_new = coffee is wrong as coffee_new will point to same memory location as coffee
coffee_new = coffee
coffee_new['price'] = 4 #this will also add price column to coffee df as both point to same memory location

coffee_new['revenue'] = coffee_new['Units Sold'] * coffee_new['price'] #adding new column based on other columns


In [ ]:
#OLYMPICS DATA ->
bios_new = bios.copy()
bios_new['height_category'] = bios_new['height_cm'].apply(lambda x: 'short' if x<160 else 'average' if x<185 else 'unknown' if pd.isna(x) else 'tall')

bios_new['normalised_name'] = bios_new['name'].str.lower().apply(unidecode)
bios_new = bios_new.sort_values(by='normalised_name').reset_index(drop=True)
 
bios_new = bios_new.drop(columns=['normalised_name']) #we dont need the normalised_name column now
bios_new 

MERGING & CONCATENATION

In [68]:
bios_new

,athlete_id,name,born_date,born_city,born_region,born_country,NOC,height_cm,weight_kg,died_date,height_category,normalised_name
0,115084,Abdul Rashid,1979-02-15,Khanewal,Punjab,PAK,Pakistan,184.0,70.0,NaN,average,abdul rashid
1,19960,Abdul Rashid,1922-06-01,Rawalpindi,Punjab,PAK,Pakistan,169.0,91.0,1988-03-08,average,abdul rashid
2,19961,Abdul Rashid,1947-03-03,Bannu,Khyber-Pukhtunkhwa,PAK,Pakistan,170.0,75.0,2020-11-04,average,abdul rashid
3,74423,Abdul Rashid,1928-04-25,NaN,NaN,NaN,Pakistan,NaN,NaN,NaN,unknown,abdul rashid
4,24163,Abel Braga,1952-09-01,Rio de Janeiro,Rio de Janeiro,BRA,Brazil,190.0,73.0,NaN,tall,abel braga
...,...,...,...,...,...,...,...,...,...,...,...,...
145495,23111,Zygmunt Pawlas,1930-10-28,Jasienica,Śląskie,POL,Poland,179.0,72.0,2001-06-20,average,zygmunt pawlas
145496,74836,Zygmunt Siedlecki,1907-04-04,Parakhonsk,Brest,BLR,Poland,185.0,82.0,1977-08-28,tall,zygmunt siedlecki
145497,56542,Zygmunt Smalcerz,1941-06-08,Bestwinka,Śląskie,POL,Poland,153.0,52.0,NaN,short,zygmunt smalcerz
145498,74870,Zygmunt Weiss,1903-04-04,Warszawa (Warsaw),Mazowieckie,POL,Poland,180.0,78.0,1977-06-04,average,zygmunt weiss
